In [ ]:
import gt4py as gt
from gt4py import gtscript
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

mpl.rcParams["font.size"] = 16

In [ ]:
# grid size
nx = 10
ny = 10
nz=1

backend='numpy'

# storage shape
shape = (nx, ny, nz)

# default origin (trust it for now!)
default_origin = (0, 0,0)

# allocate an empty storage
h_new = gt.storage.empty(backend, default_origin, shape, dtype=float)

# allocate a storage filled with zeros
v = gt.storage.zeros(backend, default_origin, shape, dtype=float)

# allocate a storage filled with ones
u = gt.storage.ones(backend, default_origin, shape, dtype=float)

# create storages out of numpy.ndarrays
h = gt.storage.from_array(np.random.rand(*shape), backend, default_origin)
h_now = gt.storage.from_array(np.random.rand(*shape), backend, default_origin)

hMidx = gt.storage.from_array(np.zeros((nx-1, ny, nz)), backend, default_origin)
hMidy = gt.storage.from_array(np.zeros((nx, ny-1, nz)), backend, default_origin)

dx = gt.storage.from_array(np.ones((nx-1, ny, nz)), backend, default_origin)
dy = gt.storage.from_array(np.ones((nx,ny-1, nz)), backend, default_origin)

dxc = gt.storage.from_array(np.ones((nx-2, ny-2, nz)), backend, default_origin)
dyc = gt.storage.from_array(np.ones((nx-2, ny-2, nz)), backend, default_origin)

In [ ]:
def test_function(
    u: gtscript.Field[float], 
    v: gtscript.Field[float], 
    h: gtscript.Field[float],
    h_new: gtscript.Field[float],
    hMidx: gtscript.Field[float],
    hMidy: gtscript.Field[float],
    dx: gtscript.Field[float],
    dy: gtscript.Field[float],
    *,
    dt: float):
    from __gtscript__ import PARALLEL, computation, interval
    
    with computation(PARALLEL), interval(...):
            hu = h * u
            # Mid-point value for h along x
            hMidx = 0.5 * (h[1, 0,0] + h[0,0,0]) - \
                    0.5 * dt / dx[0,0,0] * (hu[1,0,0] - hu[0,0,0])
            
            # Mid-point value for h along y
            hMidy = 0.5 * (h[0,1,0] + h[0,0,0]) - \
                    0.5 * self.dt / self.dy1[0,0,0] * (hv1[0,1,0] - hv1[0,0,0])
            
            # Mid-point value for h along x
            #hMidx = 0.5 * (h[1:,1:-1] + h[:-1,1:-1]) - \
            #     0.5 * self.dt / self.dx[:,1:-1] * (hu[1:,1:-1] - hu[:-1,1:-1])

            # Mid-point value for h along y
            #hMidy = 0.5 * (h[1:-1,1:] + h[1:-1,:-1]) - \
            #     0.5 * self.dt / self.dy1[1:-1,:] * (hv1[1:-1,1:] - hv1[1:-1,:-1])
        

def test_function_2(
    h: gtscript.Field[float],
    h_new: gtscript.Field[float],
    huMidx: gtscript.Field[float],
    hvMidy: gtscript.Field[float],
    dxc: gtscript.Field[float],
    dyc: gtscript.Field[float],
    *,
    dt: float):
    from __gtscript__ import PARALLEL, computation, interval

    with computation(PARALLEL), interval(...):
            hnew = h[0,0,0] - \
               dt / dxc * (huMidx[1,0,0] - huMidx[0,0,0]) - \
               dt / dy1c * (hvMidy[0,1,0] - hvMidy[0,0,0])
        
        
        

In [ ]:
test_it = gtscript.stencil(definition=test_function, backend=backend)

In [ ]:
test_it(u,v,h,h_new, hMidx, hMidy,
    dx,
    dy,
    dt=1.0, origin=(0,0,0), domain=(nx-1, ny-1, nz))

In [ ]:
plt.imshow(np.asarray(hMidx[:,:,0]))